Packages laden

In [1]:
import numpy as np
from scipy.optimize import curve_fit, differential_evolution
import matplotlib.pyplot as plt
from matplotlib import rcParams
import pandas as pd

Funktionen für die Durchführung der Optimierung und Plotparameter definieren

In [ ]:
# Definieren des Modells (Kosinussatz)
def model(c2, a, b, c1, gamma0):
    cos_arg = (a**2 + b**2 - (c1 + c2)**2) / (2 * a * b)
    # Begrenze cos_arg auf den Bereich [-1, 1], um Fehler bei arccos zu vermeiden
    cos_arg = np.clip(cos_arg, -1, 1)
    gamma = np.arccos(cos_arg) + gamma0
    return gamma

# Fehlerfunktion definieren
def error_function(params, c2_data, gamma_data):
    return np.sum((model(c2_data, *params) - gamma_data) ** 2)

# Schriftart und Layout einstellen
rcParams.update({
    "text.usetex": True,  # Damit LaTeX-Schrift verwendet wird
    "font.family": "sans-serif",
    "font.sans-serif": ["Latin Modern Sans"],
    "font.size": 10,      # Schöne Standardgröße
    "figure.dpi": 300,
    "figure.figsize": (6.0, 4.0),  # ca. 15cm breit -> perfekte Breite für A4-Seite in LaTeX
    "savefig.bbox": "tight"        # Speichert das Bild ohne überflüssigen Rand
})

Messwerte einlesen

In [3]:
points = pd.read_csv('../data/Messpunkte_Übertragungsfunktion_Gelenk_0.csv',sep=';')
circle_center = [811.769, -643.152,  1164.023]
points_centered = points[['x', 'y', 'z']].values - circle_center
point_zero = points_centered[0]

Berechnung der Winkel zwischen den Messpunkten und Berechnung von C2

In [4]:

# Punktweises Skalarprodukt
dot_products = np.sum(points_centered[1:,:] * point_zero, axis=1)

# Normen berechnen
norms_points = np.linalg.norm(points_centered[1:,:], axis=1)
norm_point_zero = np.linalg.norm(point_zero)

# Winkel berechnen
angles = np.arccos(dot_products / (norms_points * norm_point_zero))

# Encoderwerte und Winkel in DataFrame speichern
df_angles_encoder = pd.DataFrame(columns=['angle', 'encoder'])
df_angles_encoder['angle'] = angles
df_angles_encoder.iloc[0:,1] = points.iloc[1:,4]
df_angles_encoder['angle'] = np.where(
    df_angles_encoder['encoder'] < 2503,
    df_angles_encoder['angle'] * -1,
    df_angles_encoder['angle']
)
df_angles_encoder['encoder'] = df_angles_encoder['encoder']/10000

# Konvertiere des Dataframes in zwei numpy Arrays
gamma_list = df_angles_encoder['angle'].tolist()
c2_list = df_angles_encoder['encoder'].tolist()
# Konvertiere die Listen in numpy Arrays
gamma_data = np.array(gamma_list)
c2_data = np.array(c2_list)

Ermittlung der Initialwerte durch globale Optimierung

In [5]:
# Festlegung der Grenzen für die unterschiedlichen Parameter
bounds = [
    (0, 3),  # a
    (0, 3),  # b
    (0, 3),  # c1
    (-10, 10)  # gamma0
]

# Globale Optimierung durchführen
result = differential_evolution(
    error_function,
    bounds,
    args=(c2_data, gamma_data),
    maxiter=1000000  # Höhere Anzahl von Iterationen für bessere Präzision
)


a_est, b_est, c1_est, gamma0_est = result.x

# Anfangsschätzungen für die Parameter (anpassen falls nötig)
initial_guess = [a_est, b_est, c1_est, gamma0_est]  # [a, b, c1, gamma0]


Lokale Optimierung mit Hilfe der Winkel- und Encoderwerte

In [6]:
# Ermittlung der Parameter mit curve_fit
# Verwende die Schätzungen aus der globalen Optimierung als Startwerte
params, covariance = curve_fit(
    model,
    c2_data,
    gamma_data,
    p0=initial_guess,
    method='trf',  # 'trf' unterstützt bounds
    max_nfev=1000000,  # Erhöhen der maximalen Funktionsauswertungen
    ftol=1e-15,     # Toleranzen für höhere Präzision
    xtol=1e-15,
    gtol=1e-15
)

# # Extrahiere die geschätzten Parameter
a_est, b_est, c1_est, gamma0_est = params

# Berechne die Modellvorhersagen
gamma_pred = model(c2_data, *params)

# Berechne die Residuen
residuals = gamma_data - gamma_pred

# Fehlermaße berechnen
MSE = np.mean(residuals**2)  # Mean Squared Error
RMSE = np.sqrt(MSE)  # Root Mean Squared Error

# Gib die Ergebnisse aus
print("Geschätzte Parameter:")
print(f"a = {a_est}")
print(f"b = {b_est}")
print(f"c1 = {c1_est}")
print(f"γ0 = {gamma0_est}\n")
print("Fehlermaße:")
print(f"RMSE (Root Mean Squared Error): {RMSE}")

Geschätzte Parameter:
a = 0.24500503065987772
b = 0.7870930379036711
c1 = 0.5418426905233961
γ0 = -1.4353144860992972

Fehlermaße:
RMSE (Root Mean Squared Error): 0.0001900729491947611


Plotten des Fits und der Residuen jedes Messpunktes

In [7]:

# Erster Plot: Messdaten und Modell
plt.figure()
plt.scatter(c2_data, np.rad2deg(gamma_data), label='Messdaten')
c2_fit = np.linspace(min(c2_data), max(c2_data), 1000)
gamma_fit = model(c2_fit, *params)
plt.plot(c2_fit, np.rad2deg(gamma_fit), 'r-', label='Angepasstes Modell')
plt.xlabel(r'$c_2$ in m')
plt.ylabel(r'Winkel $\gamma$ in $^\circ$')
plt.legend()
plt.savefig('../results/Diagramme/Übertragungsfunktion_Gelenk_0_Plot.pdf')  # <<< PDF speichern
plt.close()

# Zweiter Plot: Residuen
plt.figure()
plt.scatter(c2_data, np.rad2deg(residuals))
plt.axhline(0, color='red', linestyle='--')
plt.xlabel(r'$c_2$ in m')
plt.ylabel('Residuen in $^\circ$')
plt.savefig('../results/Diagramme/Residuen_Plot_Übertragungsfunktion_Gelenk_0.pdf')  # <<< PDF speichern
plt.close()